# Plots
## Accuracy für KNN 

In [2]:
import numpy as np
import pandas as pnd

In [10]:
import csv
import seaborn as sb
from scipy import spatial
from collections import Counter
from scipy.stats import mode
from functions.PCA import pca
from pathlib import Path
import os

dataset = 1
match dataset:
    case 1:
        test = 'fashion-mnist_test.csv'
        train = 'fashion-mnist_train.csv'
    case 2: 
        test = "mnist_test.csv"
        train = "mnist_train.csv"
testdata = pnd.read_csv(test)
traindata = pnd.read_csv(train)

testdata_pixel = testdata.drop(testdata.columns[0], axis=1).to_numpy()
traindata_pixel = traindata.drop(traindata.columns[0], axis=1).to_numpy()
label_train = traindata[traindata.columns[0]].to_numpy()
label_test = testdata[testdata.columns[0]].to_numpy()

class_names = np.array(["T-shirt / Top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"])

folder_path = 'C:/Users/awark/Bioinfo Projekt/Results/'  # Path to the "results" folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pnd.read_csv(file_path)

knnallk = pnd.read_csv('C:/Users/awark/Bioinfo Projekt/Results/out.csv')
knnallk

for i in range(14):
    accuracy = np.sum(knnallk[i:] == label_test)
    print(accuracy)

ValueError: Unable to coerce to Series, length must be 14: given 10000